# Lab 7.1: Most Popular Superhero

## Tổng quan bài tập
**Đề bài**: Hãy hoàn thiện các phần `[...]` để hoàn thiện đoạn code và giải quyết bài toán theo yêu cầu. Bạn sẽ cần viết một chương trình để dựa theo tập dữ liệu để đưa ra siêu tên của siêu anh hùng có số lượng kết nối nhiều nhất, và tính được xem trung bình một siêu anh hùng có bao nhiêu kết nối.

## Tài nguyên tham khảo

Bạn có thể tải tập Dataset tại [link sau](https://drive.google.com/drive/folders/1_f81WTm1fCHG4HMUWM7JOnqVvqfofsSs?usp=sharing). Sau đó đưa lên Google Drive và kết nối với Colab là có thể sử dụng được. Tập dữ liệu gồm 2 file như sau:

1. `Marvel_Names.csv`

File .csv chứa thông tin về các siêu anh hùng với cấu trúc như sau:
- `Id`: Id của siêu anh hùng
- `Name`: Tên của siêu anh hùng đó.

2. `Marvel_Graph.txt`

File .txt chứa các thông tin về kết nối giữa các siêu anh hùng, sẽ có dạng như sau: `1602 3889 1535 6038 533 3986`

Dữ liệu này nghĩa là siêu anh hùng với ID là `1602` có kết nối đến các siêu anh hùng `3889 1535 6038 533 3986`.


Ngoài ra, bạn có thể tham khảo các video sau trong trường hợp chưa hiểu cách làm bài Lab:
- [Find the Most Popular Superhero in a Social Graph](https://funix.udemy.com/course/taming-big-data-with-apache-spark-hands-on/learn/lecture/3726116#questions)


# Cài đặt Spark trên Google Colab

Để có thể sử dụng Spark trên môi trường Google Colab thì bạn sẽ cần cài đặt một số thành phần sau:
- Java 8
- Spark Binary
- findspark

In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf spark-3.5.0-bin-hadoop3.tgz
!pip install -q findspark

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,678 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,333 kB]
Get:13 https://ppa.launchpadcontent.net/c2d4u.team/c2

Sau đó, bạn sẽ cần khai báo cho hệ thống các đường dẫn cho các thành phần vừa cài.

In [3]:
import os
import findspark

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

findspark.init()

# Kết nối với Google Drive

Để lấy dữ liệu từ các Dataset, bạn sẽ phải lưu file dữ liệu lên Google Drive. Sau đó kết nối Colab đến Google Drive của bạn và lấy được các file dữ liệu.

In [4]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


# Most Popular Superhero

Bạn sẽ cần khởi tạo 1 SparkSesson để có thể bắt đầu Spark.

In [5]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

conf = SparkConf() \
    .setMaster('local') \
    .setAppName('MostPopularSuperhero')

spark = SparkSession.builder.config(conf=conf).getOrCreate()
sc = spark.sparkContext

Đọc dữ liệu từ Dataset

In [6]:
schema = StructType([
		 StructField("id", StringType()),
		 StructField("name", StringType())
])
MARVELNAMES_PATH = "/content/gdrive/My Drive/DEP303/Marvel_Names.csv"
names = spark.read \
		.schema(schema) \
		.csv(MARVELNAMES_PATH)
names.show()

MARVELGRAPH_PATH = "/content/gdrive/My Drive/DEP303/Marvel_Graph.txt"
lines = spark.read.text(MARVELGRAPH_PATH)
lines.show()

+---+--------------------+
| id|                name|
+---+--------------------+
|  1|24-HOUR MAN/EMMANUEL|
|  2|3-D MAN/CHARLES CHAN|
|  3|    4-D MAN/MERCURIO|
|  4|             8-BALL/|
|  5|                   A|
|  6|               A'YIN|
|  7|        ABBOTT, JACK|
|  8|             ABCISSA|
|  9|                ABEL|
| 10|ABOMINATION/EMIL BLO|
| 11|ABOMINATION | MUTANT|
| 12|         ABOMINATRIX|
| 13|             ABRAXAS|
| 14|          ADAM 3,031|
| 15|             ABSALOM|
| 16|ABSORBING MAN/CARL C|
| 17|ABSORBING MAN | MUTA|
| 18|                ACBA|
| 19|ACHEBE, REVEREND DOC|
| 20|            ACHILLES|
+---+--------------------+
only showing top 20 rows

+--------------------+
|               value|
+--------------------+
|5988 748 1722 375...|
|5989 4080 4264 44...|
|5982 217 595 1194...|
|5983 1165 3836 43...|
|5980 2731 3712 15...|
|5981 3569 5353 40...|
|5986 2658 3712 26...|
|5987 2614 5716 17...|
|5984 590 4898 745...|
|5985 3233 2254 21...|
|6294 4898 1127 32...|
|270

Tạo một Dataframe để tính số lượng kết nối của một siêu anh hùng, ví dụ:
```
+----+-----------+
|  id|connections|
+----+-----------+
| 691|          6|
|1159|         11|
|3959|        142|
|1572|         35|
+----+-----------+
```

Lưu ý rằng một id của siêu anh hùng có thể xuất hiện nhiều lần nên bạn sẽ cần tính tổng, để trường id không có các dữ liệu trùng lặp.

In [13]:
def connection_count(line_value):
  return len(line_value.rstrip().split(" "))-1
def marvel_id_in_line(line_value):
  return line_value.split(" ")[0]
spark.udf.register("connection_count_udf", connection_count, IntegerType())
spark.udf.register("get_id", marvel_id_in_line, StringType())
connections = lines.withColumn("connectionCount", func.expr("connection_count_udf(value)")) \
                   .withColumn("id", func.expr("get_id(value)")) \
                   .groupBy("id") \
                   .agg(func.sum("connectionCount").alias("connections"))

connections.show()

+----+-----------+
|  id|connections|
+----+-----------+
| 691|          6|
|1159|         11|
|3959|        142|
|1572|         35|
|2294|         14|
|1090|          4|
|3606|        171|
|3414|          7|
| 296|         17|
|4821|         16|
|2162|         41|
|1436|          9|
|1512|         11|
|6194|         14|
|6240|         11|
| 829|         37|
|2136|          6|
|5645|         20|
|2069|        263|
| 467|          0|
+----+-----------+
only showing top 20 rows



Tính số kết nối trung bình của một siêu anh hùng.

In [14]:
avg_df = connections.select(func.avg("connections").alias("AVG_CONNECTIONS"))

avg_df.show()

+-----------------+
|  AVG_CONNECTIONS|
+-----------------+
|51.88621646623497|
+-----------------+



Join 2 Dataframe để lấy được dữ liệu về tên của siêu anh hùng

In [15]:
join_expr = connections.id == names.id
joined_df = connections \
              .join(names, join_expr, "inner") \
              .drop(names.id)

joined_df.show()

+----+-----------+--------------------+
|  id|connections|                name|
+----+-----------+--------------------+
| 691|          6|BRANDEIS, LIZ PARRIS|
|1159|         11|       CONTONI, PAUL|
|3959|        142|NIGHTMARE/EDVARD HAB|
|1572|         35|DORNOVA-BLONSKY, NAD|
|2294|         14|      GUTHRIE, TYLER|
|1090|          4|             CLUSTER|
|3606|        171| MELTER/BRUNO HORGAN|
|3414|          7|MAN-SPIDER CLONE | M|
| 296|         17|       AYALA, AWILDA|
|4821|         16|               SALLA|
|2162|         41|                 GOG|
|1436|          9|        DELGADO, LUZ|
|1512|         11|        DIETZ, SUSAN|
|6194|         14|               WENDY|
|6240|         11|          WILD BLOOD|
| 829|         37|    CALDRONE, JAVIER|
|2136|          6|       GILMORE, BERT|
|5645|         20|            TARTARUS|
|2069|        263|GARGOYLE II/ISAAC CH|
| 467|          0|        BERSERKER II|
+----+-----------+--------------------+
only showing top 20 rows



Tìm tên siêu anh hùng có nhiều kết nối nhất

In [26]:
mostPopular = joined_df.sort(func.desc("connections")).first()
# mostPopular.show()
print(
	mostPopular[2]
	+ " is the most popular superhero with "
	+ str(mostPopular[1])
	+ " co-appearances."
)

CAPTAIN AMERICA is the most popular superhero with 1933 co-appearances.
